In [1]:
N_REFRIGERATORS = 100
SIMULATION_YEARS = 2
DATA_INTERVAL_MINUTES = 10
FAILURE_RATE = 0.05
DEGRADATION_PERIOD_DAYS = 60
LABELING_WINDOW_DAYS = 30

In [2]:
TIMESTEPS_PER_HOUR = 60 // 10
HOURS_PER_DAY = 24
SEQUENCE_DAYS = 3
SEQUENCE_TIMESTEPS = SEQUENCE_DAYS * HOURS_PER_DAY * TIMESTEPS_PER_HOUR
N_FEATURES = 6

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import joblib
import random
from datetime import datetime, timedelta, timezone

In [4]:
MODEL_FILE = '/content/sequences.h5'
SCALER_FILE = '/content/scaler.joblib'

In [5]:
def generate_sample_sequence(is_failing=False):
    """
    Generates a sample sequence of raw data for inference.
    If is_failing is True, it simulates data from a degrading fridge.
    """
    print(f"\nGenerating a sample {'FAILING' if is_failing else 'HEALTHY'} data sequence...")
    raw_data = []
    timestamp = datetime.now(timezone.utc) - timedelta(minutes=SEQUENCE_TIMESTEPS * 10)
    temp_setpoint = 4.0

    for i in range(SEQUENCE_TIMESTEPS):
        degradation_factor = 0.0
        if is_failing:
            # Simulate being in the last 3 days of a 60-day degradation period
            degradation_factor = 1 - ((SEQUENCE_TIMESTEPS - i) / (DEGRADATION_PERIOD_DAYS * HOURS_PER_DAY * TIMESTEPS_PER_HOUR))
            degradation_factor = max(0.5, degradation_factor) # Ensure degradation is significant

        temp_current = temp_setpoint + random.uniform(-0.5, 0.5) + (degradation_factor * 2.5)
        compressor_on = 1 if temp_current > temp_setpoint else 0
        if random.random() < (degradation_factor * 0.3):
            compressor_on = 1

        raw_data.append({
            "temperature_current_c": temp_current,
            "compressor_on": compressor_on,
            "door_open": 0,
            "energy_consumption_kwh": (0.005 + degradation_factor * 0.005) if compressor_on else 0.0001,
            "voltage_v": random.uniform(225.0, 245.0),
            "ambient_temp_c": 22.5
        })
        timestamp += timedelta(minutes=10)

    print(raw_data)

    return raw_data

def run_inference(raw_data_sequence):
    """
    Loads the model and scaler, preprocesses the data, and returns a prediction.
    """
    # --- 1. Load Model and Scaler ---
    try:
        model = keras.models.load_model(MODEL_FILE)
        scaler = joblib.load(SCALER_FILE)
    except Exception as e:
        print(f"Error loading model or scaler: {e}")
        return

    # --- 2. Pre-process the Input Data ---
    # Convert to DataFrame
    df = pd.DataFrame(raw_data_sequence)
    feature_columns = [
        'temperature_current_c', 'compressor_on', 'door_open',
        'energy_consumption_kwh', 'voltage_v', 'ambient_temp_c'
    ]
    df = df[feature_columns] # Ensure correct column order

    # Scale the features using the loaded scaler
    scaled_features = scaler.transform(df)

    # Reshape into the 3D format the LSTM expects: (1, timesteps, features)
    sequence = np.array([scaled_features])

    if sequence.shape != (1, SEQUENCE_TIMESTEPS, N_FEATURES):
        print(f"Error: Input data has incorrect shape {sequence.shape}")
        return

    # --- 3. Run Prediction ---
    prediction_proba = model.predict(sequence)[0][0]

    # --- 4. Display Results ---
    risk_score = float(prediction_proba)
    is_failure_imminent = bool(risk_score > 0.5)

    print("\n--- Inference Result ---")
    print(f"Model Prediction: '{'Failure Imminent' if is_failure_imminent else 'Normal Operation'}'")
    print(f"Failure Risk Score: {risk_score:.4f}")
    print("----------------------")


if __name__ == "__main__":
    # --- Test with a HEALTHY sample ---
    healthy_sequence = generate_sample_sequence(is_failing=False)
    run_inference(healthy_sequence)

    # --- Test with a FAILING sample ---
    failing_sequence = generate_sample_sequence(is_failing=True)
    run_inference(failing_sequence)


Generating a sample HEALTHY data sequence...
[{'temperature_current_c': 4.323288349749101, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.005, 'voltage_v': 231.29916162106937, 'ambient_temp_c': 22.5}, {'temperature_current_c': 4.115436541412239, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.005, 'voltage_v': 227.94811419492507, 'ambient_temp_c': 22.5}, {'temperature_current_c': 4.371991788980193, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.005, 'voltage_v': 243.97651797015337, 'ambient_temp_c': 22.5}, {'temperature_current_c': 3.9204904405245973, 'compressor_on': 0, 'door_open': 0, 'energy_consumption_kwh': 0.0001, 'voltage_v': 229.8091500659442, 'ambient_temp_c': 22.5}, {'temperature_current_c': 4.3604044791472845, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.005, 'voltage_v': 244.30049608510154, 'ambient_temp_c': 22.5}, {'temperature_current_c': 4.027042473741853, 'compressor_on': 1, 'door_open': 0, 'energy_co


--- Inference Result ---
Model Prediction: 'Normal Operation'
Failure Risk Score: 0.0000
----------------------

Generating a sample FAILING data sequence...
[{'temperature_current_c': 6.124107074815529, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.00975, 'voltage_v': 231.92347520045578, 'ambient_temp_c': 22.5}, {'temperature_current_c': 6.56911501630215, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.009750578703703704, 'voltage_v': 228.53590656433977, 'ambient_temp_c': 22.5}, {'temperature_current_c': 6.347572073041272, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.009751157407407408, 'voltage_v': 243.11012774822825, 'ambient_temp_c': 22.5}, {'temperature_current_c': 6.072527417624759, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 0.00975173611111111, 'voltage_v': 225.01484481868727, 'ambient_temp_c': 22.5}, {'temperature_current_c': 6.584467288550564, 'compressor_on': 1, 'door_open': 0, 'energy_consumption_kwh': 